In [1]:
from tensorflow.python.client import device_lib
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

Num GPUs Available:  1


In [2]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [3]:
from tensorflow import keras
from tensorflow.keras import layers, models, metrics
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.applications import *
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dense, Input, Dropout, GlobalMaxPooling2D 
from sklearn.model_selection import train_test_split

In [4]:
pwd

'D:\\projects\\Personal\\Fraudulent-Signature-Detection\\model\\EfficientNet'

In [5]:
from numpy import save, load
X_train=load('../../data/X_train.npy')
y_train=load('../../data/y_train.npy')
X_test=load('../../data/X_test.npy')
y_test=load('../../data/y_test.npy')

In [6]:
X_train.shape,y_train.shape

((14644, 256, 512, 3), (14644, 1))

In [7]:
X_test.shape,y_test.shape

((3661, 256, 512, 3), (3661, 1))

In [8]:
input_shape=(256,512,3)

In [10]:
base_model = keras.applications.EfficientNetB5(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(256, 512, 3),
    include_top=False,
)

115277824/115263384 [==============================] - 94s 1us/step


In [11]:
base_model.trainable = False


In [14]:
inputs = keras.Input(shape=(256, 512, 3))
x=inputs

In [15]:
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.3)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

In [16]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 256, 512, 3)]     0         
                                                                 
 efficientnetb5 (Functional)  (None, 8, 16, 2048)      28513527  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_1 (Dropout)         (None, 2048)              0         
                                                                 
 dense_1 (Dense)             (None, 1)                 2049      
                                                                 
Total params: 28,515,576
Trainable params: 2,049
Non-trainable params: 28,513,527
___________________________________________

In [17]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 20

In [18]:
history_efficient_net=model.fit(X_train[:5000],y_train[:5000], epochs=epochs, validation_data=(X_test[:1000],y_test[:1000]))

Epoch 1/20
157/157 [==============================] - 102s 553ms/step - loss: 0.6949 - binary_accuracy: 0.4616 - val_loss: 0.6900 - val_binary_accuracy: 0.4580
Epoch 2/20
157/157 [==============================] - 103s 656ms/step - loss: 0.6947 - binary_accuracy: 0.4656 - val_loss: 0.7151 - val_binary_accuracy: 0.4580
Epoch 3/20
157/157 [==============================] - 142s 906ms/step - loss: 0.6932 - binary_accuracy: 0.4606 - val_loss: 0.6868 - val_binary_accuracy: 0.4580
Epoch 4/20
157/157 [==============================] - 183s 1s/step - loss: 0.6893 - binary_accuracy: 0.4588 - val_loss: 0.6859 - val_binary_accuracy: 0.4580
Epoch 5/20
157/157 [==============================] - 216s 1s/step - loss: 0.6892 - binary_accuracy: 0.4636 - val_loss: 0.6893 - val_binary_accuracy: 0.4570
Epoch 6/20
157/157 [==============================] - 249s 2s/step - loss: 0.6910 - binary_accuracy: 0.4656 - val_loss: 0.6871 - val_binary_accuracy: 0.4580
Epoch 7/20
157/157 [=============================

In [19]:
model.save("../saved_models/eff_net_base.h5")

D:\projects\Jupyter\gpu-env\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
